# Trinomial Classification of News Dataset

In [1]:
import pandas
import matplotlib.pyplot as plt
from collections import Counter
import string
import numpy as np
from sklearn.utils import class_weight
from tensorflow.keras import layers
from tensorflow.keras import losses
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow import keras
from sklearn.utils import class_weight

2025-05-23 21:05:42.342767: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-05-23 21:05:42.362435: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-05-23 21:05:42.381945: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-05-23 21:05:42.388166: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-05-23 21:05:42.404794: I tensorflow/core/platform/cpu_feature_guar

In [2]:
#removing the stop words (commonly used words that do not add much meaning to a sentence)

## Yet to actually apply this function to the data

import nltk
from nltk.corpus import stopwords

stop = set(stopwords.words("english"))

def remove_stopwords(text):
    #removes stop words
    filtered_words = [word.lower() for word in text.split() if word.lower() not in stop]
    return " ".join(filtered_words)


def remove_punctuation(text):
    translator = str.maketrans("", "", string.punctuation)
    return text.translate(translator)

In [3]:
wf = pandas.read_csv("./news_sentiment_analysiscsv.csv")
wf = wf.sample(frac=1).reset_index(drop=True)
wf

,Source,Author,Title,Description,URL,Published At,Sentiment,Type
0,americanbankingnews,ABMN Staff,"Sei Investments Co. Acquires 250,354 Shares of...","Sei Investments Co. Acquires 250,354 Shares of...",https://www.americanbankingnews.com/2024/07/18...,2024-07-18T10:44:44+00:00,neutral,Entertainment
1,paradisepost,Associated Press,"Rally shooter had photos of Trump, Biden and o...",The man who tried to assassinate Trump had pho...,https://www.paradisepost.com/2024/07/18/rally-...,2024-07-18T17:33:07+00:00,negative,General
2,koreaherald,Kim Jun-hong,Tmax Tibero’s research center recognized top R...,Tmax Group announced Thursday that the researc...,https://www.koreaherald.com/view.php?ud=202407...,2024-07-18T05:35:13+00:00,positive,Science
3,newswise,Space Telescope Science Institute (STScI),Vivid Portrait of Interacting Galaxies Marks W...,The James Webb Space Telescope operates around...,http://www.newswise.com/articles/view/813984/?...,2024-07-12T14:30:43+00:00,positive,Science
4,fwweekly,Kristian Lin,Dandelion: Once in a Generation,Why isn’t KiKi Layne a bigger name? It’s still...,https://www.fwweekly.com/2024/07/16/dandelion-...,2024-07-16T21:35:59+00:00,positive,General
...,...,...,...,...,...,...,...,...
3495,Polygon,Michael McWhertor,"Overwatch 2’s next hero, Juno, revealed — and ...",Image: Blizzard Entertainment The support hero...,https://www.polygon.com/24200684/overwatch-2-s...,2024-07-18T16:00:48+00:00,neutral,Entertainment
3496,krdo,Emily Arseneau,One in 30 million orange lobster saved from de...,"PUEBLO, Colo. (KRDO) - The Red Lobster in Pueb...",https://krdo.com/news/2024/07/16/one-in-30-mil...,2024-07-16T21:08:58+00:00,positive,General
3497,mailonsunday,NaN,This is Carlos Alcaraz's time. His brutal - ye...,RIATH AL-SAMARRAI ON CENTRE COURT: Five weeks ...,https://www.dailymail.co.uk/sport/tennis/artic...,2024-07-14T20:22:01+00:00,neutral,Sports
3498,dvidshub,Sgt. James Lundy,18th Field Artillery Brigade receives treatmen...,A Soldier from the 18th Field Artillery Brigad...,https://www.dvidshub.net/image/8536712/18th-fi...,2024-07-18T19:54:59+00:00,neutral,Health


In [4]:
def count_unique_word(wf):
    count = Counter()
    for word in wf['Sentiment']:
        count[word] += 1
    return count

counter = count_unique_word(wf)

In [5]:
print(counter.most_common())

[('positive', 2134), ('neutral', 789), ('negative', 577)]


In [6]:

for i in range(len(wf)):
    if wf['Sentiment'][i] == counter.most_common()[0][0]:
        wf['Sentiment'][i] = 0
    elif wf['Sentiment'][i] == counter.most_common()[1][0]:
        wf['Sentiment'][i] = 1
    elif wf['Sentiment'][i] == counter.most_common()[2][0]:
        wf['Sentiment'][i] = 2

/tmp/ipykernel_14320/317023626.py:5: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  wf['Sentiment'][i] = 1
/tmp/ipykernel_14320/317023626.py:7: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values

In [7]:
def count_unique_word(wf_training):
    count = Counter()
    for i in range(len(wf_training)):
        for word in wf_training.iloc[i]['Description'].split():
            count[word] += 1
    return count

counter = count_unique_word(wf)


for i in range(len(wf['Description'])):
    wf['Description'][i] = remove_punctuation(wf['Description'][i])
    wf['Description'][i] = remove_stopwords(wf['Description'][i])

for i in range(len(wf) - 1, -1, -1):
        if not (isinstance(wf.iloc[i]['Description'], str)):
                wf = wf.drop(i)


counter = count_unique_word(wf)




/tmp/ipykernel_14320/73436677.py:12: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  wf['Description'][i] = remove_punctuation(wf['Description'][i])
/tmp/ipykernel_14320/73436677.py:13: FutureWarning: ChainedAssignmentError: behaviour will cha

In [8]:
num_unique_words = len(counter)



wf_training = wf[:int(len(wf) * 0.8)]
wf_testing = wf[int(len(wf) * 0.8):]



wf_testing.reset_index(inplace=True)
wf_testing.drop('index', axis=1, inplace=True)
wf_testing



wf_training.reset_index(inplace=True)
wf_training.drop('index', axis=1, inplace=True)
wf_training


training_sentences = wf_training.Description.to_numpy()
#training labels
training_labels = wf_training.Sentiment.to_numpy()
validation_sentences = wf_testing.Description.to_numpy()
#training labels
validation_labels = wf_testing.Sentiment.to_numpy()




tokenizer = Tokenizer(num_words=num_unique_words)
tokenizer.fit_on_texts(training_sentences)


word_index = tokenizer.word_index




training_sequences = tokenizer.texts_to_sequences(training_sentences)
validation_sequences = tokenizer.texts_to_sequences(validation_sentences)


max_length = 100
training_padded = pad_sequences(training_sequences, maxlen=max_length, padding="post", truncating="post")
validation_padded = pad_sequences(validation_sequences, maxlen=max_length, padding="post", truncating="post")
training_padded.shape, validation_padded.shape


/tmp/ipykernel_14320/2728969571.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  wf_testing.drop('index', axis=1, inplace=True)
/tmp/ipykernel_14320/2728969571.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  wf_training.drop('index', axis=1, inplace=True)


((2800, 100), (700, 100))

In [17]:
print(num_unique_words)

22069


In [9]:
#LSTM Model



model = keras.models.Sequential()

#input layer
#embedding layer converts the words into vectors of fixed size
model.add(layers.Embedding(num_unique_words, 32))
#dropout rate drops out a certain percentage of input data so that the
model.add(layers.Bidirectional(layers.LSTM(64, dropout=0.2, recurrent_dropout=0.2)))
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(3, activation='softmax'))
model.summary()


I0000 00:00:1748048745.158612   14320 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1748048745.236349   14320 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1748048745.236420   14320 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1748048745.241659   14320 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1748048745.241712   14320 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:0

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional (Bidirectional)   │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [10]:
loss = 'sparse_categorical_crossentropy'
optim = keras.optimizers.Adam(learning_rate=0.001) #from 0.001
metrics = ["accuracy"]

model.compile(loss=loss, optimizer=optim, metrics=metrics)

In [11]:
training_labels = np.array(training_labels).astype(np.int64)
validation_labels = np.array(validation_labels).astype(np.int64)

In [12]:

class_weights = class_weight.compute_class_weight(
    class_weight='balanced',
    classes=np.unique(training_labels),
    y=training_labels
)
class_weights = dict(enumerate(class_weights))

In [13]:
print(class_weights)

{0: 0.5522682445759369, 1: 1.4744602422327542, 2: 1.9566736547868624}


In [14]:
wf_training['Description'].iloc[3]

'james webb space telescope operates around clock frequently astonishing researchers highly detailed incredibly precise infrared images data wavelengths light lie beyond eyes see largely reach level detail webb began taking science observations july 12 2022'

In [15]:
print(counter)

Counter({'de': 839, '2024': 817, 'first': 795, 'shares': 719, 'july': 587, '8230': 581, 'la': 576, 'health': 571, 'post': 501, 'appeared': 480, 'new': 460, 'entertainment': 446, 'report': 445, 'technology': 433, 'sports': 420, 'business': 414, 'us': 410, '8211': 398, 'free': 383, 'stock': 379, 'inc': 378, 'science': 362, 'general': 348, 'quarter': 331, 'company': 312, 'marketscreenercom': 311, 'di': 302, 'air': 284, 'en': 276, 'according': 256, 'said': 243, 'get': 235, 'der': 227, 'del': 219, 'el': 216, 'force': 212, 'recent': 209, 'team': 202, 'technologies': 199, 'die': 197, 'holdings': 194, 'one': 185, 'owned': 184, 'group': 177, '1st': 172, 'securities': 171, '—': 170, 'le': 169, 'company8217s': 169, 'news': 169, '18': 169, 'un': 169, 'sciences': 168, '8230the': 166, 'photo': 166, 'services': 165, 'que': 164, 'e': 162, 'medical': 160, 'il': 156, '16': 152, 'people': 151, 'announced': 150, 'two': 149, 'reports': 142, 'per': 142, 'commission': 139, 'und': 139, 'also': 138, 'army': 13

In [16]:
model.fit(training_padded, training_labels, epochs=4, validation_data=(validation_padded, validation_labels), class_weight=class_weights, batch_size=32, verbose=2)

Epoch 1/4
88/88 - 67s - 758ms/step - accuracy: 0.5111 - loss: 0.9912 - val_accuracy: 0.7043 - val_loss: 0.7418
Epoch 2/4
88/88 - 62s - 700ms/step - accuracy: 0.7343 - loss: 0.5922 - val_accuracy: 0.6514 - val_loss: 0.7356
Epoch 3/4
88/88 - 61s - 698ms/step - accuracy: 0.8918 - loss: 0.2772 - val_accuracy: 0.8086 - val_loss: 0.5434
Epoch 4/4
88/88 - 58s - 660ms/step - accuracy: 0.9596 - loss: 0.1166 - val_accuracy: 0.8414 - val_loss: 0.5118
